<a href="https://colab.research.google.com/github/gabyperez/evaluacionmodelos/blob/main/Vectorizer_skLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 16.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#Descargo los escenarios

In [ ]:
#@title Se realizan las consultas
query = "Realizar fumigación para controlar plagas"
#query = "Recortar ramas de la planta"
#query = "Distribuir agua en los cultivos"
#query = "Recolectar los tomates maduros"
#query = "Erradicar vegetación indeseada"

titulos_escenarios  = ["Eliminar las malezas", "Quitar las malas hierbas", "Controlar las plagas", "Despuntar las inflorescencias",
             "Regar las plantas de tomate", "Controlar las enfermedades bacterianas", "Prevención de enfermedades fungosas",
             "Cosechar los tomates de forma manual", "Realizar el podado de las plantas", "Controlar las plagas e insectos", "Regar las plántulas de tomate",
             "Cosechar los tomates en racimos", "Controlar las enfermedades virales", "Realizar la poda de forma manual"]
print("query", query)

query Realizar fumigación para controlar plagas


In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar el modelo de español de spaCy
nlp = spacy.load("es_core_news_sm")

# trabajo las oraciones sacando los stopwords y lemmatizando
frases_minusculas = [oracion.lower() for oracion in titulos_escenarios]
frases_procesadas = [nlp(frase) for frase in frases_minusculas]
tokens_lematizados = [[token.lemma_ for token in frase if not token.is_stop] for frase in frases_procesadas]
print(tokens_lematizados)

# Convertir los tokens lematizados de cada frase a texto nuevamente
textos_frases = [" ".join(tokens) for tokens in tokens_lematizados]

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Aplicar el vectorizador TF-IDF a los textos de las frases lematizadas
tfidf_matrix = tfidf_vectorizer.fit_transform(textos_frases)

# Verificar la forma de la matriz TF-IDF
print("Forma de la matriz TF-IDF:", tfidf_matrix.shape)  # 19 escenarios y 31 terminos


# Verificar el vocabulario generado por el vectorizador
print("Vocabulario generado por el vectorizador:", tfidf_vectorizer.get_feature_names_out())


[['eliminar', 'maleza'], ['quitar', 'mala', 'hierba'], ['controlar', 'plaga'], ['despuntar', 'inflorescencia'], ['regar', 'planta', 'tomate'], ['controlar', 'enfermedad', 'bacteriana'], ['prevención', 'enfermedad', 'fungosa'], ['cosechar', 'tomate', 'forma', 'manual'], ['podado', 'planta', 'fumigación'], ['controlar', 'plaga', 'insecto'], ['regar', 'plántula', 'tomate'], ['cosechar', 'tomate', 'racimo'], ['controlar', 'enfermedad', 'viral'], ['poda', 'forma', 'manual']]
Forma de la matriz TF-IDF: (14, 26)
Vocabulario generado por el vectorizador: ['bacteriana' 'controlar' 'cosechar' 'despuntar' 'eliminar' 'enfermedad'
 'forma' 'fumigación' 'fungosa' 'hierba' 'inflorescencia' 'insecto' 'mala'
 'maleza' 'manual' 'plaga' 'planta' 'plántula' 'poda' 'podado'
 'prevención' 'quitar' 'racimo' 'regar' 'tomate' 'viral']


In [ ]:
#@title identifico la consulta que quiero hacer
# trabajo las oraciones sacando los stopwords y lemmatizando
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#convierto el titulo nuevo
query_minuscula = query.lower()
query_procesada = nlp(query_minuscula)
query_lematizada = [token.lemma_ for token in query_procesada if not token.is_stop]
print(query_lematizada)
texto_consulta = " ".join(query_lematizada)

texto_en_buscar_embedding = tfidf_vectorizer.transform([texto_consulta])


#comparo con coseno el titulo nuevo con los otros escenarios
coseno_scores = cosine_similarity(texto_en_buscar_embedding, tfidf_matrix)


#los ordeno y los enumero
indices_ordenados_maxmin = np.argsort(-coseno_scores).tolist()[0]
valores_coseno = coseno_scores[0].tolist()


respuestas_mayores = []
for indice, indice_coseno in enumerate(indices_ordenados_maxmin):
    #print("indice", indice_coseno, "valor: ", valores_coseno[indice_coseno])
    elemento = [indice_coseno, "indice real", indice_coseno+1,  titulos_escenarios[indice_coseno], valores_coseno[indice_coseno]]
    respuestas_mayores.append(elemento)
[print(item) for item in respuestas_mayores]




['fumigación', 'controlar', 'plaga']
[2, 'indice real', 3, 'Controlar las plagas', 0.7431690203273478]
[9, 'indice real', 10, 'Controlar las plagas e insectos', 0.55230019277431]
[8, 'indice real', 9, 'Realizar el podado de las plantas fumigación', 0.4035498316068679]
[5, 'indice real', 6, 'Controlar las enfermedades bacterianas', 0.2249227315895555]
[12, 'indice real', 13, 'Controlar las enfermedades virales', 0.2249227315895555]
[0, 'indice real', 1, 'Eliminar las malezas', 0.0]
[1, 'indice real', 2, 'Quitar las malas hierbas', 0.0]
[3, 'indice real', 4, 'Despuntar las inflorescencias', 0.0]
[4, 'indice real', 5, 'Regar las plantas de tomate', 0.0]
[6, 'indice real', 7, 'Prevención de enfermedades fungosas', 0.0]
[7, 'indice real', 8, 'Cosechar los tomates de forma manual', 0.0]
[10, 'indice real', 11, 'Regar las plántulas de tomate', 0.0]
[11, 'indice real', 12, 'Cosechar los tomates en racimos', 0.0]
[13, 'indice real', 14, 'Realizar la poda de forma manual', 0.0]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]